In [4]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time
from datetime import datetime
from scipy.stats import linregress
from pprint import pprint

# Import citipy to determine the cities based on latitude and longitude
from citipy import citipy

In [8]:
url = "https://data.cityofchicago.org/resource/68nd-jvt3.json"
data = requests.get(url)
len(data.json())
#pprint(data.json())


1000

In [34]:
path1 = "Resources/vehicles.csv"
path2 = "Resources/crashes.csv"

In [12]:
vehicles_df = pd.read_csv(path1)
vehicles_df.keys()

C:\Users\Connor Thomas\AppData\Local\Temp\ipykernel_14912\1583178177.py:1: DtypeWarning: Columns (19,21,40,41,42,44,48,49,50,53,55,58,59,61,71) have mixed types. Specify dtype option on import or set low_memory=False.
  vehicles_df = pd.read_csv(path1)


Index(['CRASH_UNIT_ID', 'CRASH_RECORD_ID', 'RD_NO', 'CRASH_DATE', 'UNIT_NO',
       'UNIT_TYPE', 'NUM_PASSENGERS', 'VEHICLE_ID', 'CMRC_VEH_I', 'MAKE',
       'MODEL', 'LIC_PLATE_STATE', 'VEHICLE_YEAR', 'VEHICLE_DEFECT',
       'VEHICLE_TYPE', 'VEHICLE_USE', 'TRAVEL_DIRECTION', 'MANEUVER',
       'TOWED_I', 'FIRE_I', 'OCCUPANT_CNT', 'EXCEED_SPEED_LIMIT_I', 'TOWED_BY',
       'TOWED_TO', 'AREA_00_I', 'AREA_01_I', 'AREA_02_I', 'AREA_03_I',
       'AREA_04_I', 'AREA_05_I', 'AREA_06_I', 'AREA_07_I', 'AREA_08_I',
       'AREA_09_I', 'AREA_10_I', 'AREA_11_I', 'AREA_12_I', 'AREA_99_I',
       'FIRST_CONTACT_POINT', 'CMV_ID', 'USDOT_NO', 'CCMC_NO', 'ILCC_NO',
       'COMMERCIAL_SRC', 'GVWR', 'CARRIER_NAME', 'CARRIER_STATE',
       'CARRIER_CITY', 'HAZMAT_PLACARDS_I', 'HAZMAT_NAME', 'UN_NO',
       'HAZMAT_PRESENT_I', 'HAZMAT_REPORT_I', 'HAZMAT_REPORT_NO',
       'MCS_REPORT_I', 'MCS_REPORT_NO', 'HAZMAT_VIO_CAUSE_CRASH_I',
       'MCS_VIO_CAUSE_CRASH_I', 'IDOT_PERMIT_NO', 'WIDE_LOAD_I',
       '

In [30]:
vehicles_df['USDOT_NO'].unique()

array([nan, '3374027', '116433', ..., 2811363.0, 301539.0, 2520879.0],
      dtype=object)

In [33]:
vehicles_data_df = vehicles_df.loc[:,['CRASH_RECORD_ID','CRASH_DATE','UNIT_TYPE',
                                      'NUM_PASSENGERS','MAKE','MODEL',
                                      'VEHICLE_YEAR','VEHICLE_TYPE','TRAVEL_DIRECTION',
                                      'MANEUVER','OCCUPANT_CNT','EXCEED_SPEED_LIMIT_I']]
vehicles_data_df = vehicles_data_df.rename(columns={'CRASH_RECORD_ID':"Crash Record ID",
                                                    'CRASH_DATE':"Crash Date",
                                                    'UNIT_TYPE':"Vehicle Type",
                                                    'NUM_PASSENGERS':"Number of Passengers",
                                                    'MAKE':"Make",
                                                    'MODEL':"Model",
                                                    'VEHICLE_YEAR':"Vehicle Year",
                                                    'VEHICLE_TYPE':"Vehicle Type",
                                                    'TRAVEL_DIRECTION':"Travel Direction",
                                                    'MANEUVER':"Maneuver",
                                                    'OCCUPANT_CNT':"Number of Occupants",
                                                    'EXCEED_SPEED_LIMIT_I':"Exceeded Speed Limit"})
vehicles_data_df

,Crash Record ID,Crash Date,Vehicle Type,Number of Passengers,Make,Model,Vehicle Year,Vehicle Type,Travel Direction,Maneuver,Number of Occupants,Exceeded Speed Limit
0,24ddf9fd8542199d832e1c223cc474e5601b356f1d77a6...,01/22/2020 06:25:00 AM,DRIVER,NaN,INFINITI,UNKNOWN,2017.0,PASSENGER,N,STRAIGHT AHEAD,1.0,NaN
1,81dc0de2ed92aa62baccab641fa377be7feb1cc47e6554...,09/28/2019 03:30:00 AM,DRIVER,NaN,HONDA,CIVIC,2016.0,PASSENGER,N,STRAIGHT AHEAD,1.0,NaN
2,81dc0de2ed92aa62baccab641fa377be7feb1cc47e6554...,09/28/2019 03:30:00 AM,PARKED,NaN,TOYOTA,YARIS,2010.0,UNKNOWN/NA,N,PARKED,0.0,NaN
3,81dc0de2ed92aa62baccab641fa377be7feb1cc47e6554...,09/28/2019 03:30:00 AM,PARKED,NaN,GENERAL MOTORS CORPORATION (GMC),SIERRA,2008.0,UNKNOWN/NA,N,PARKED,0.0,NaN
4,af84fb5c8d996fcd3aefd36593c3a02e6e7509eeb27568...,04/13/2020 10:50:00 PM,DRIVER,NaN,BUICK,ENCORE,NaN,PASSENGER,W,STRAIGHT AHEAD,1.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
1412211,6c26e8407561884de71818e40d831baa8933a3f1d51081...,01/21/2023 12:59:00 PM,DRIVER,NaN,SCION,TC,2008.0,PASSENGER,N,CHANGING LANES,1.0,NaN
1412212,6c26e8407561884de71818e40d831baa8933a3f1d51081...,01/21/2023 12:59:00 PM,DRIVER,NaN,JEEP,CHEROKEE,2018.0,SPORT UTILITY VEHICLE (SUV),N,STRAIGHT AHEAD,1.0,NaN
1412213,03a2dc75ecec7d98ff335864b6d6e40a68e179a972e31c...,01/21/2023 09:00:00 PM,DRIVER,NaN,DODGE,DART,2016.0,PASSENGER,N,STRAIGHT AHEAD,1.0,NaN
1412214,03a2dc75ecec7d98ff335864b6d6e40a68e179a972e31c...,01/21/2023 09:00:00 PM,DRIVER,2.0,TOYOTA,RAV4,2017.0,PASSENGER,N,SLOW/STOP IN TRAFFIC,3.0,NaN


In [13]:
crashes_df = pd.read_csv(path2)
crashes_df.keys()

Index(['CRASH_RECORD_ID', 'RD_NO', 'CRASH_DATE_EST_I', 'CRASH_DATE',
       'POSTED_SPEED_LIMIT', 'TRAFFIC_CONTROL_DEVICE', 'DEVICE_CONDITION',
       'WEATHER_CONDITION', 'LIGHTING_CONDITION', 'FIRST_CRASH_TYPE',
       'TRAFFICWAY_TYPE', 'LANE_CNT', 'ALIGNMENT', 'ROADWAY_SURFACE_COND',
       'ROAD_DEFECT', 'REPORT_TYPE', 'CRASH_TYPE', 'INTERSECTION_RELATED_I',
       'NOT_RIGHT_OF_WAY_I', 'HIT_AND_RUN_I', 'DAMAGE', 'DATE_POLICE_NOTIFIED',
       'PRIM_CONTRIBUTORY_CAUSE', 'SEC_CONTRIBUTORY_CAUSE', 'STREET_NO',
       'STREET_DIRECTION', 'STREET_NAME', 'BEAT_OF_OCCURRENCE',
       'PHOTOS_TAKEN_I', 'STATEMENTS_TAKEN_I', 'DOORING_I', 'WORK_ZONE_I',
       'WORK_ZONE_TYPE', 'WORKERS_PRESENT_I', 'NUM_UNITS',
       'MOST_SEVERE_INJURY', 'INJURIES_TOTAL', 'INJURIES_FATAL',
       'INJURIES_INCAPACITATING', 'INJURIES_NON_INCAPACITATING',
       'INJURIES_REPORTED_NOT_EVIDENT', 'INJURIES_NO_INDICATION',
       'INJURIES_UNKNOWN', 'CRASH_HOUR', 'CRASH_DAY_OF_WEEK', 'CRASH_MONTH',
       'LA

In [14]:
crashes_df['CRASH_DAY_OF_WEEK'].unique()

array([2, 4, 6, 1, 7, 3, 5], dtype=int64)